In [1]:
import pickle
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Part 1 : Listing Movies to Query

In [7]:
movies = pd.read_csv('data/tmdb_5000_movies.csv')
movies.drop(['homepage', 'keywords','original_language','overview','release_date','spoken_languages', \
             'status','title','tagline','vote_count'\
            ], \
            axis=1, \
            inplace=True \
           )

In [8]:
credits = pd.read_csv('data/tmdb_5000_credits.csv')
credits = credits[credits.cast != '[]']
credits.drop(['title', 'crew'], axis=1, inplace=True)
credits['cast_id'] = credits['cast'].apply(lambda row: list(set(pd.read_json(row)['id'])))

In [9]:
frames = pd.DataFrame()
new_df = pd.DataFrame()

for idx, film in credits.iterrows():
    cast_df = pd.DataFrame(eval(credits['cast'][idx]))
    cast_df['credits'] = idx
    cast_df = cast_df.drop(['character','order', 'credit_id', 'cast_id'],axis = 1)  
    
    frames = [new_df, cast_df]
    new_df = pd.concat(frames, join = 'outer', ignore_index=True)

In [10]:
discount_old = credits['cast_id'].apply(pd.Series).stack().value_counts()
discount_old = list(discount_old[discount_old > 4].index.astype(int))

In [11]:
nodes_df = new_df['credits'].groupby([new_df.gender, new_df.id, new_df.name]).apply(list).reset_index()
nodes_df = nodes_df[nodes_df['gender'].isin(['1','2'])]
discount_1 = nodes_df['id'].tolist()
discount = [x for x in discount_old if x in discount_1]

In [12]:
credits['cast_id'] = credits['cast_id'].apply(lambda x: [y for y in x if y in discount])
credits['edges'] = credits['cast_id'].apply(lambda x: list(itertools.combinations(x, 2)))

In [13]:
discarded_movies = set()

for idx, movie in credits.iterrows():
    if len(movie['edges']) == 0:
        discarded_movies.add(movie['movie_id'])

print(len(discarded_movies)) 

663


In [14]:
movies = movies[~movies['id'].isin(discarded_movies)]
movies['bechdel'] = ''
movies.head()

,budget,genres,id,original_title,popularity,production_companies,production_countries,revenue,runtime,vote_average,bechdel
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,Avatar,150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2787965087,162.0,7.2,
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,Pirates of the Caribbean: At World's End,139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",961000000,169.0,6.9,
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,Spectre,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",880674609,148.0,6.3,
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,The Dark Knight Rises,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1084939099,165.0,7.6,
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,John Carter,43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",284139100,132.0,6.1,


## Part 2 : Requesting Ratings

In [15]:
import requests
import tmdbsimple as tmdb

In [16]:
tmdb.API_KEY = 'd3e76e53927bba96fdae8b86fb9d4570'

In [17]:
movie = tmdb.Movies(movies['id'][0]).info()
idx = movie['imdb_id'][2:]
idx

'0499549'

In [18]:
r = requests.get('http://bechdeltest.com/api/v1/getMovieByImdbId', params={'imdbid': idx})

In [19]:
r.json()['rating']

'2'

In [23]:
for idx, row in movies_bechdel[3305:].iterrows():
#for idx, row in movies.iterrows():   
    if(idx%100 == 0):
        print("Processing film nbr {}".format(idx))
        print("Advancement : {}%".format(idx/len(movies)*100))
    
    try:
        movie = tmdb.Movies(row['id']).info()
        row['imdb_id'] = movie['imdb_id'][2:]
    except requests.exceptions.HTTPError:
        print("Movie {} with id {} cannot be found on TMBD".format(row['original_title'], row['id']))
        movies['bechdel'][idx] = -1
        continue
    
    r = requests.get('http://bechdeltest.com/api/v1/getMovieByImdbId', params={'imdbid': row['imdb_id']})
    
    try:
        movies_bechdel['bechdel'][idx] = r.json()['rating']
    except KeyError:
        print("Movie {} with id {} isn't rated".format(row['original_title'], row['id']))
        movies_bechdel['bechdel'][idx] = -1

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Movie Novocaine with id 20794 isn't rated


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Movie The Business of Strangers with id 31064 isn't rated
Movie The Wild Bunch with id 576 isn't rated
Movie The Wackness with id 13990 isn't rated
Movie The First Great Train Robbery with id 11583 isn't rated
Movie Beastmaster 2: Through the Portal of Time with id 27549 isn't rated
Movie The 5th Quarter with id 59728 isn't rated
Movie The Greatest with id 32395 isn't rated
Movie Come Early Morning with id 7547 isn't rated
Processing film nbr 3500
Advancement : 84.54106280193237%
Movie Lucky Break with id 35944 isn't rated
Movie Surfer, Dude with id 13827 isn't rated
Movie 11:14 with id 9282 isn't rated
Movie Men of War with id 38940 isn't rated
Movie Don McKay with id 35689 isn't rated
Movie Deadfall with id 97614 isn't rated
Movie A Shine of Rainbows with id 37206 isn't rated
Movie The Hit List with id 58626 isn't rated
Movie Emma with id 183894 cannot be found on TMBD


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


Movie L'Auberge espagnole with id 1555 isn't rated
Movie Song One with id 244783 isn't rated
Movie Freaky Deaky with id 168027 isn't rated
Movie Trade Of Innocents with id 135595 isn't rated
Movie Stiff Upper Lips with id 89861 isn't rated
Movie Sunshine State with id 57022 isn't rated
Movie Crossover with id 14351 isn't rated
Movie Diary of a Mad Black Woman with id 16186 isn't rated
Movie Shine with id 7863 isn't rated
Movie Iris with id 11889 isn't rated
Movie Mambo Italiano with id 321 isn't rated
Movie Wonderland with id 4997 isn't rated
Movie Harvard Man with id 27451 isn't rated
Movie Salvation Boulevard with id 51995 isn't rated
Movie The Ten with id 13173 isn't rated
Movie A Room for Romeo Brass with id 22908 isn't rated
Movie Saint Ralph with id 25248 isn't rated
Movie Miss Julie with id 230266 isn't rated
Movie Jackass: The Movie with id 9012 isn't rated
Movie The Return of the Pink Panther with id 11843 isn't rated
Movie The Elephant Man with id 1955 isn't rated
Movie House

Movie Tusk with id 246403 isn't rated
Movie Compadres with id 317930 isn't rated
Movie Creature with id 50942 cannot be found on TMBD
Movie Brave New Girl with id 34099 isn't rated
Movie Fort McCoy with id 281230 isn't rated
Movie Chain Letter with id 38033 isn't rated
Movie Just Looking with id 100975 isn't rated
Movie The Eclipse with id 39563 isn't rated
Movie Demonic with id 234212 isn't rated
Movie The Deported with id 170480 isn't rated
Movie Tanner Hall with id 71866 isn't rated
Movie Never Back Down 2: The Beatdown with id 70006 isn't rated
Movie Point Blank with id 26039 isn't rated
Movie Enter the Dangerous Mind with id 176077 isn't rated
Movie Something Wicked with id 260947 isn't rated
Movie AWOL-72 with id 342502 isn't rated
Movie Iguana with id 191229 isn't rated
Movie Chicago Overcoat with id 43213 isn't rated
Movie Barry Munday with id 44594 isn't rated
Movie Pocketful of Miracles with id 248 isn't rated
Movie Close Range with id 325173 isn't rated
Movie Amnesiac with i

Movie Two Girls and a Guy with id 32456 isn't rated
Movie Who Killed the Electric Car? with id 13508 isn't rated
Movie The Broken Hearts Club: A Romantic Comedy with id 22597 isn't rated
Movie Bubba Ho-tep with id 9707 isn't rated
Movie All the Real Girls with id 13132 isn't rated
Movie Dream with the Fishes with id 47686 isn't rated
Movie Blue Car with id 46989 isn't rated
Movie The Battle of Shaker Heights with id 21413 isn't rated
Movie Lisa Picard Is Famous with id 206412 isn't rated
Movie The Perfect Host with id 66195 isn't rated
Movie Speedway Junky with id 46727 isn't rated
Movie The Last Big Thing with id 55180 isn't rated
Movie The Specials with id 29015 isn't rated
Movie 16 to Life with id 91122 isn't rated
Movie Special with id 13856 isn't rated
Processing film nbr 4400
Advancement : 106.28019323671498%
Movie Sparkler with id 219716 isn't rated
Movie The Helix... Loaded with id 43630 isn't rated
Movie In Her Line of Fire with id 56491 isn't rated
Movie The Jimmy Show with i

In [85]:
movies[3300:]

,budget,genres,id,original_title,popularity,production_companies,production_countries,revenue,runtime,vote_average,bechdel
3481,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",19556,Miss March,9.371534,"[{""name"": ""The Jacobson Company"", ""id"": 3638}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",0,89.0,5.4,-1
3482,5500000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 35, ""name...",231576,Wish I Was Here,15.015271,"[{""name"": ""Double Feature Films"", ""id"": 215}, ...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",5483299,120.0,6.4,3
3483,6000000,"[{""id"": 35, ""name"": ""Comedy""}]",16172,Simply Irresistible,5.696491,"[{""name"": ""New Regency Pictures"", ""id"": 10104}...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",4398989,95.0,5.5,-1
3484,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10402, ""...",13403,Hedwig and the Angry Inch,4.254220,"[{""name"": ""New Line Cinema"", ""id"": 12}, {""name...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",0,95.0,7.4,3
3485,6000000,"[{""id"": 28, ""name"": ""Action""}]",15797,Only the Strong,5.411243,"[{""name"": ""Davis Films"", ""id"": 14175}, {""name""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",3283371,99.0,6.9,-1
3487,6000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 36, ""name...",13537,Shattered Glass,6.272570,"[{""name"": ""Lions Gate Films"", ""id"": 35}, {""nam...","[{""iso_3166_1"": ""CA"", ""name"": ""Canada""}, {""iso...",2944752,94.0,6.6,
3488,6000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 80, ""nam...",20794,Novocaine,4.148253,"[{""name"": ""Numb Gums Production Inc."", ""id"": 3...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2025238,95.0,6.0,
3489,0,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 53, ""name...",31064,The Business of Strangers,1.369649,"[{""name"": ""i5 Films"", ""id"": 26927}, {""name"": ""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",0,84.0,5.7,
3490,6244087,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 37, ""...",576,The Wild Bunch,14.970654,"[{""name"": ""Warner Brothers/Seven Arts"", ""id"": ...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",638641,145.0,7.6,
3491,6000000,"[{""id"": 18, ""name"": ""Drama""}]",13990,The Wackness,7.161051,"[{""name"": ""SBK Pictures"", ""id"": 11665}, {""name...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2899975,99.0,6.2,


In [86]:
movies.head()

,budget,genres,id,original_title,popularity,production_companies,production_countries,revenue,runtime,vote_average,bechdel
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,Avatar,150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2787965087,162.0,7.2,2
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,Pirates of the Caribbean: At World's End,139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",961000000,169.0,6.9,3
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,Spectre,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",880674609,148.0,6.3,1
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,The Dark Knight Rises,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1084939099,165.0,7.6,1
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,John Carter,43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",284139100,132.0,6.1,3


In [26]:
with open('data/bechdel.pkl', 'wb') as f: 
    pickle.dump(movies_bechdel, f)

In [25]:
movies_bechdel['bechdel'].value_counts()

-1    1646
3     1358
1      679
2      273
0      178
         6
Name: bechdel, dtype: int64

In [2]:
with open('data/bechdel.pkl', 'rb') as pkl_file: 
    movies_bechdel = pickle.load(pkl_file)
    
movies_bechdel.head()

,budget,genres,id,original_title,popularity,production_companies,production_countries,revenue,runtime,vote_average,bechdel
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,Avatar,150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2787965087,162.0,7.2,2
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,Pirates of the Caribbean: At World's End,139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",961000000,169.0,6.9,3
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,Spectre,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",880674609,148.0,6.3,1
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,The Dark Knight Rises,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1084939099,165.0,7.6,1
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,John Carter,43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",284139100,132.0,6.1,3


In [22]:
movies_bechdel[3304:]

,budget,genres,id,original_title,popularity,production_companies,production_countries,revenue,runtime,vote_average,bechdel
3485,6000000,"[{""id"": 28, ""name"": ""Action""}]",15797,Only the Strong,5.411243,"[{""name"": ""Davis Films"", ""id"": 14175}, {""name""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",3283371,99.0,6.9,-1
3487,6000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 36, ""name...",13537,Shattered Glass,6.272570,"[{""name"": ""Lions Gate Films"", ""id"": 35}, {""nam...","[{""iso_3166_1"": ""CA"", ""name"": ""Canada""}, {""iso...",2944752,94.0,6.6,
3488,6000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 80, ""nam...",20794,Novocaine,4.148253,"[{""name"": ""Numb Gums Production Inc."", ""id"": 3...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2025238,95.0,6.0,
3489,0,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 53, ""name...",31064,The Business of Strangers,1.369649,"[{""name"": ""i5 Films"", ""id"": 26927}, {""name"": ""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",0,84.0,5.7,
3490,6244087,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 37, ""...",576,The Wild Bunch,14.970654,"[{""name"": ""Warner Brothers/Seven Arts"", ""id"": ...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",638641,145.0,7.6,
3491,6000000,"[{""id"": 18, ""name"": ""Drama""}]",13990,The Wackness,7.161051,"[{""name"": ""SBK Pictures"", ""id"": 11665}, {""name...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2899975,99.0,6.2,
3492,6000000,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 12, ""n...",11583,The First Great Train Robbery,12.434688,"[{""name"": ""United Artists"", ""id"": 60}, {""name""...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}]",0,110.0,6.6,
3494,6000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",27549,Beastmaster 2: Through the Portal of Time,1.478505,"[{""name"": ""Les Films 21"", ""id"": 1113}, {""name""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",869325,107.0,4.6,
3495,6000000,"[{""id"": 18, ""name"": ""Drama""}]",59728,The 5th Quarter,0.215464,"[{""name"": ""McCreary Entertainment"", ""id"": 7953...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",399611,97.0,4.7,
3497,6000000,"[{""id"": 18, ""name"": ""Drama""}]",32395,The Greatest,6.903186,"[{""name"": ""Irish Dreamtime"", ""id"": 597}, {""nam...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",0,99.0,6.2,
